In [153]:
import Scrapers.ackerman_scraper as ack_scraper
import numpy as np

In [154]:
df = ack_scraper.dataframe_ackerman_auction()

{1185: '183 Apr 7, 2018 (NY)', 1187: '183A Apr 21, 2018 (HK)', 1188: '184 May 16, 2018 (NY)', 1190: '184A Jun 9, 2018 (HK)', 1192: '185 Jun 20, 2018 (NY)', 1184: '182A Mar 24, 2018 (HK)', 1196: '186 Sep 7, 2018 (NY)', 1197: '185A Sep 22, 2018 (HK)', 1165: '175A Sep 23, 2017 (HK)', 1168: '177 Oct 13, 2017 (NY)', 1202: '187 Oct 12, 2018 (NY)', 1172: '178 Nov 18, 2017 (NY)', 1173: '177A Dec 9, 2017 (HK)', 1174: '179 Dec 13, 2017 (NY)', 1177: '181A Jan 27, 2018 (HK)', 1179: '181 Feb 9, 2018 (NY)', 1163: '176 Sep 9, 2017 (NY)', 1181: '182 Mar 10, 2018 (NY)', 1170: '176A Nov 4, 2017 (HK)'}
1185 183 Apr 7, 2018 (NY)
1187 183A Apr 21, 2018 (HK)
1188 184 May 16, 2018 (NY)
1190 184A Jun 9, 2018 (HK)
1192 185 Jun 20, 2018 (NY)
1184 182A Mar 24, 2018 (HK)
1196 186 Sep 7, 2018 (NY)
1197 185A Sep 22, 2018 (HK)
1165 175A Sep 23, 2017 (HK)
1168 177 Oct 13, 2017 (NY)
1202 187 Oct 12, 2018 (NY)
1172 178 Nov 18, 2017 (NY)
1173 177A Dec 9, 2017 (HK)
1174 179 Dec 13, 2017 (NY)
1177 181A Jan 27, 2018 (HK)
1

In [155]:
df.head()

,Auction.Date,Auction.ID,Auction.Name,BottleQuantity,BottleSummary,Conditions,Designation,LotNo,Producer,ProducerAndDesignation,Size,SoldAmount,Vintage,Wine
0,None,1185,"183 Apr 7, 2018 (NY)",1,1 magnum,1x1 magnum owc lscl,Hommage a Jacques Perrin,1,Chateau de Beaucastel,"Hommage a Jacques Perrin, Chateau de Beaucastel",magnum,1860.0,1989,Chateauneuf du Pape
1,None,1185,"183 Apr 7, 2018 (NY)",3,3 bottles,1lscl,Hommage a Jacques Perrin,2,Chateau de Beaucastel,"Hommage a Jacques Perrin, Chateau de Beaucastel",bottle,1736.0,2001,Chateauneuf du Pape
2,None,1185,"183 Apr 7, 2018 (NY)",1,1 magnum,1x1 magnum owc,Hommage a Jacques Perrin,2,Chateau de Beaucastel,"Hommage a Jacques Perrin, Chateau de Beaucastel",magnum,1736.0,2001,Chateauneuf du Pape
3,None,1185,"183 Apr 7, 2018 (NY)",1,1 bottle,3.75cm bc cuc gsvl lscl outstanding color and ...,None,3,Chateau Rayas,Chateau Rayas,bottle,13640.0,1929,Chateauneuf du Pape
4,None,1185,"183 Apr 7, 2018 (NY)",1,1 bottle,7.5cm bc excellent color lscl lstl,None,4,Chateau Rayas,Chateau Rayas,bottle,4712.0,1959,Chateauneuf du Pape


In [156]:
df.shape

(31962, 14)

In [157]:
#convert to dollars
HKDUSD = 0.1277
def toUSD(row):
    if 'HK' in row['Auction.Name']:
       return row['SoldAmount'] * HKDUSD
    else:
       return row['SoldAmount'] 

df[ 'SoldAmountUSD' ] = df.apply( toUSD, axis = 1 )
    
# compute unit price by bottle
df['UnitPriceUSD'] = df[ 'SoldAmountUSD' ] / df[ 'BottleQuantity' ]

# bottle sizes only for now
bottle_df = df[ df["BottleSummary"].str.contains( "bottle") ]
bottle_df.shape

(27224, 16)

In [158]:
bottle_grouped = df.groupby(['Wine', 'Vintage', "Producer"]).UnitPriceUSD.agg( ['min', 'mean', 'max', "count"] )

In [159]:
bottle_grouped = bottle_grouped.reset_index()
bottle_grouped = bottle_grouped.sort_values('count', ascending = False)

In [160]:
mouton_grouped = bottle_grouped[ bottle_grouped[ "Wine"].str.contains( "Mouton") ]
mouton = bottle_df[ bottle_df[ "Wine"].str.contains( "Mouton") ]

In [164]:
#SOME ERRORS HERE - unit prices still too crazy
mouton[ mouton['UnitPriceUSD'] > 10000 ]

,Auction.Date,Auction.ID,Auction.Name,BottleQuantity,BottleSummary,Conditions,Designation,LotNo,Producer,ProducerAndDesignation,Size,SoldAmount,Vintage,Wine,SoldAmountUSD,UnitPriceUSD
592,None,1187,"183A Apr 21, 2018 (HK)",1,1 bottle,,None,518,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,272800.0,1989,Chateau Mouton Rothschild,34836.56,34836.560
593,None,1187,"183A Apr 21, 2018 (HK)",1,1 bottle,lbsl,None,518,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,272800.0,1990,Chateau Mouton Rothschild,34836.56,34836.560
594,None,1187,"183A Apr 21, 2018 (HK)",1,1 bottle,ts,None,518,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,272800.0,1991,Chateau Mouton Rothschild,34836.56,34836.560
595,None,1187,"183A Apr 21, 2018 (HK)",1,1 bottle,,None,518,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,272800.0,1992,Chateau Mouton Rothschild,34836.56,34836.560
596,None,1187,"183A Apr 21, 2018 (HK)",2,2 bottles,1 balthus 1lbsl 1lgsl 1vlscl 2sdc,None,518,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,272800.0,1993,Chateau Mouton Rothschild,34836.56,17418.280
597,None,1187,"183A Apr 21, 2018 (HK)",1,1 bottle,bsl excellent color and condition nc nl scc vhs,None,518,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,272800.0,1970,Chateau Mouton Rothschild,34836.56,34836.560
598,None,1187,"183A Apr 21, 2018 (HK)",1,1 bottle,excellent color and condition lbsl lscl nc vhs,None,518,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,272800.0,1971,Chateau Mouton Rothschild,34836.56,34836.560
599,None,1187,"183A Apr 21, 2018 (HK)",1,1 bottle,bsl excellent color and condition lgsl sdc vhs,None,518,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,272800.0,1972,Chateau Mouton Rothschild,34836.56,34836.560
600,None,1187,"183A Apr 21, 2018 (HK)",1,1 bottle,excellent color and condition nc vhs vlbsl vlscl,None,518,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,272800.0,1973,Chateau Mouton Rothschild,34836.56,34836.560
601,None,1187,"183A Apr 21, 2018 (HK)",1,1 bottle,excellent color and condition lbsl lgsl nc nl ts,None,518,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,272800.0,1974,Chateau Mouton Rothschild,34836.56,34836.560


In [162]:
mouton_grouped.head()

,Wine,Vintage,Producer,min,mean,max,count
4012,Chateau Mouton Rothschild,1982,Chateau Mouton Rothschild,992.000000,11021.391657,75700.56,56
4016,Chateau Mouton Rothschild,1986,Chateau Mouton Rothschild,806.000000,9954.086851,75700.56,54
4030,Chateau Mouton Rothschild,2000,Chateau Mouton Rothschild,1766.346400,13404.348195,75700.56,49
4028,Chateau Mouton Rothschild,1998,Chateau Mouton Rothschild,232.500000,16059.554867,75700.56,30
4026,Chateau Mouton Rothschild,1996,Chateau Mouton Rothschild,452.833333,18142.373930,75700.56,27
